<a href="https://colab.research.google.com/github/ridhijs/STEM-Away-MLTeam7/blob/MLTeam7c/Data/combining_datasets_and_implementing_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 757kB 5.2MB/s 
     |████████████████████████████████| 890kB 17.4MB/s 
     |████████████████████████████████| 1.1MB 32.6MB/s 
     |████████████████████████████████| 3.0MB 50.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=34e7fc1c15f8bf700c956f3d1a61a6a835210b3ff192feef4bdb08aa5d68d3fc
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import pandas as pd
import numpy as np
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

from transformers import DistilBertTokenizer

In [3]:
#load in all the files

df_7c_1 = pd.read_csv('https://raw.githubusercontent.com/ridhijs/STEM-Away-MLTeam7/0c760690c8098d54c8e18465a3583f493b349260/Data/DF1_team7c.csv')
df_7c_2 = pd.read_csv('https://raw.githubusercontent.com/ridhijs/STEM-Away-MLTeam7/0c760690c8098d54c8e18465a3583f493b349260/Data/DF2_team7c.csv')

In [4]:
df_7b_1 = pd.read_csv('https://raw.githubusercontent.com/ridhijs/STEM-Away-MLTeam7/MLteam7b/DF1_team7b.csv')
df_7b_2 = pd.read_csv('https://raw.githubusercontent.com/ridhijs/STEM-Away-MLTeam7/MLteam7b/DF2_team7b.csv')

In [5]:
df_7a_1 = pd.read_csv('https://raw.githubusercontent.com/ridhijs/STEM-Away-MLTeam7/mlteam7a/Data/DF1_team7a.csv')
df_7a_2 = pd.read_csv('https://raw.githubusercontent.com/ridhijs/STEM-Away-MLTeam7/mlteam7a/Data/DF2_team7a.csv')

In [6]:
#create a new dataframe for the first datasets and use new indexing
df_all_data_1 = pd.concat([df_7c_1, df_7b_1, df_7a_1], ignore_index = True)

df_all_data_1

,Title,Category,Sub-category,Post Text,URL
0,Used Sold As New - Account Review,Account Health,Account Health,Ive reviewed a lot of discussion in this forum...,https://sellercentral.amazon.com/forums/t/used...
1,Account Suspended After 5 years out of nowhere...,Account Health,Related Accounts,Our brand was suspended about 25 minutes ago. ...,https://sellercentral.amazon.com/forums/t/acco...
2,Account Suspension after Credit Card Change,Account Health,Account Health,"Hello Everyone,\nI am trying to open an Amazon...",https://sellercentral.amazon.com/forums/t/acco...
3,Our Seller account has been deactivated,Account Health,Related Accounts,"Hi,\nWe need help. Since last week our seller ...",https://sellercentral.amazon.com/forums/t/our-...
4,How to Handle Used as New Complaints Version 2,Account Health,Product Condition,This is my second thread regarding used as new...,https://sellercentral.amazon.com/forums/t/how-...
...,...,...,...,...,...
1813,Amazon Custom Program,Amazon Custom,Amazon Custom Forum,"Are you applying? Dear Seller, Amazon Custom i...",https://sellercentral.amazon.com/forums/t/amaz...
1814,"Just enabled ‘Amazon Custom’ on my products, a...",Amazon Custom,Amazon Custom Forum,"Hi there, I just enabled ‘Amazon Custom’ (for ...",https://sellercentral.amazon.com/forums/t/just...
1815,"Amazon Custom - Refunds. To do, or not to do?",Amazon Custom,Amazon Custom Forum,I am an Amazon “Custom” seller. I have had my ...,https://sellercentral.amazon.com/forums/t/amaz...
1816,Custom Order Feedback,Amazon Custom,Amazon Custom Forum,I sold a custom item recently and was wonderin...,https://sellercentral.amazon.com/forums/t/cust...


In [7]:
#create a new dataframe for the second datasets and use new indexing
df_all_data_2 = pd.concat([df_7c_2, df_7b_2, df_7a_2], ignore_index = True)

df_all_data_2

,URL,Response Text
0,https://sellercentral.amazon.com/forums/t/used...,"Tagging @Shelf_Cleaning_Capit, @Susan_Amazon"
1,https://sellercentral.amazon.com/forums/t/used...,Too much irrelevant information in the beginni...
2,https://sellercentral.amazon.com/forums/t/used...,"Thank you for the comment. Just to be clear, t..."
3,https://sellercentral.amazon.com/forums/t/acco...,Do you have any other accounts using your addr...
4,https://sellercentral.amazon.com/forums/t/acco...,Have the same issue here. Account was deactiva...
...,...,...
8225,https://sellercentral.amazon.com/forums/t/amaz...,Thanks for the input. From what I have experie...
8226,https://sellercentral.amazon.com/forums/t/amaz...,even though custom items cannot be returned as...
8227,https://sellercentral.amazon.com/forums/t/amaz...,I agree. After this initial learning curve and...
8228,https://sellercentral.amazon.com/forums/t/cust...,"Hey Gravity Glass, Your customer can absolutel..."


In [8]:
#drop the null values
df_all_data_1 = df_all_data_1.dropna()

In [9]:
df_all_data_2 = df_all_data_2.dropna()

In [10]:
#just considering the post text and category of each post
subdata1 = df_all_data_1[['Title', 'Category']]

subdata1

,Title,Category
0,Used Sold As New - Account Review,Account Health
1,Account Suspended After 5 years out of nowhere...,Account Health
2,Account Suspension after Credit Card Change,Account Health
3,Our Seller account has been deactivated,Account Health
4,How to Handle Used as New Complaints Version 2,Account Health
...,...,...
1813,Amazon Custom Program,Amazon Custom
1814,"Just enabled ‘Amazon Custom’ on my products, a...",Amazon Custom
1815,"Amazon Custom - Refunds. To do, or not to do?",Amazon Custom
1816,Custom Order Feedback,Amazon Custom


In [11]:
#let's give a label to our categorical data
subdata1['Category'] = subdata1['Category'].astype('category')
subdata1['Label'] = subdata1['Category'].cat.codes.astype('int32')

In [13]:
subdata1.sample(5)

,Title,Category,Label
1680,Custom list can not upload image,Amazon Custom,1
1444,Total BS! Listing removed for being used and s...,Amazon Custom,1
1701,What do i do if they don’t send the item back?,Amazon Custom,1
1579,How to validate IP claim for authenticity?,Amazon Custom,1
117,Account deactivated,Account Health,0


### DistilBERT

In [14]:
#DistilBert
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

#load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

### Tokenize

In [15]:
#tokenize the post titles
tokenized = subdata1['Title'].apply((lambda x: tokenizer.encode(x, add_special_tokens = True)))

In [ ]:
tokenized

0       [101, 4921, 2063, 8182, 1037, 2843, 1997, 6594...
1       [101, 2256, 4435, 2001, 6731, 2055, 2423, 2781...
2       [101, 7592, 3071, 1010, 1045, 2572, 2667, 2000...
3       [101, 7632, 1010, 2057, 2342, 2393, 1012, 2144...
4       [101, 2023, 2003, 2026, 2117, 11689, 4953, 210...
                              ...                        
1813    [101, 2024, 2017, 11243, 1029, 6203, 14939, 10...
1814    [101, 7632, 2045, 1010, 1045, 2074, 9124, 1520...
1815    [101, 1045, 2572, 2019, 9733, 1523, 7661, 1524...
1816    [101, 1045, 2853, 1037, 7661, 8875, 3728, 1998...
1817    [101, 6603, 2065, 3087, 2040, 2003, 2747, 5604...
Name: Post Text, Length: 1815, dtype: object

### Padding

In [16]:
#making all the vectors the same size by adding zeroes to the end
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
        
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

In [17]:
np.array(padded).shape

(1815, 46)

In [18]:
padded

array([[ 101, 2109, 2853, ...,    0,    0,    0],
       [ 101, 4070, 6731, ...,    0,    0,    0],
       [ 101, 4070, 8636, ...,    0,    0,    0],
       ...,
       [ 101, 9733, 7661, ...,    0,    0,    0],
       [ 101, 7661, 2344, ...,    0,    0,    0],
       [ 101, 9733, 7661, ...,    0,    0,    0]])

### Masking

In [19]:
# When we send our vectors to BERT, we don't wanna confuse it, so we tell it to
# ignore the padded we've added
#says, if an entry in padded is non-zero, place a 1 there, otherwise, keep 0 there.

attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1815, 46)

In [20]:
attention_mask

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

### Implement BERT

In [21]:
# model() runs our titles through DistilBERT and stores the results 
# in last_hidden_states

input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask = attention_mask)

In [22]:
last_hidden_states

(tensor([[[-0.2987, -0.1951, -0.2050,  ..., -0.3205,  0.0079,  0.2928],
          [ 0.1118,  0.1722, -0.1689,  ..., -0.2847,  0.3555,  0.0911],
          [ 0.0171, -0.3881,  0.1743,  ..., -0.4270, -0.3054,  0.0425],
          ...,
          [ 0.2110, -0.0167,  0.0052,  ..., -0.0079, -0.2847,  0.0301],
          [ 0.1501, -0.1457,  0.1771,  ..., -0.1131, -0.1022,  0.1141],
          [ 0.3113, -0.1613,  0.1510,  ..., -0.0960, -0.1376,  0.0273]],
 
         [[-0.2555, -0.2464, -0.0091,  ..., -0.2630,  0.1791,  0.2592],
          [ 0.2396, -0.2908,  0.0894,  ..., -0.0704,  0.1717,  0.1895],
          [-0.0816, -0.0418,  0.1850,  ..., -0.4769, -0.2043, -0.2650],
          ...,
          [-0.0447, -0.2475,  0.2535,  ...,  0.0893, -0.4130,  0.0915],
          [-0.0336, -0.2031,  0.1130,  ...,  0.1216, -0.3515,  0.1255],
          [-0.0360, -0.1804,  0.1136,  ...,  0.1059, -0.3022,  0.1682]],
 
         [[-0.2024, -0.1282, -0.1994,  ..., -0.2429, -0.2209,  0.3531],
          [ 0.1824, -0.4550,

In [23]:
 # Slice the output for the first position for all the sequences, 
 # take all hidden unit outputs. This takes our 3d tensor to the 2d 
 # tensor we want. 
 
features = last_hidden_states[0][:,0,:].numpy()

In [24]:
# Now we have a 2d numpy array containing the title embeddings of all the titles
# in our dataframe

features

array([[-0.29865476, -0.19507974, -0.20497093, ..., -0.32045913,
         0.00794015,  0.2928201 ],
       [-0.25547445, -0.24642347, -0.00914994, ..., -0.26295692,
         0.17914344,  0.25920343],
       [-0.20243658, -0.12817791, -0.19943656, ..., -0.24293612,
        -0.22086985,  0.35314402],
       ...,
       [-0.03164658, -0.125362  , -0.09875356, ..., -0.06413957,
         0.38573706,  0.3833593 ],
       [-0.25228223, -0.23297726,  0.0626864 , ..., -0.19984004,
        -0.08183997,  0.39651406],
       [-0.3352247 , -0.2128325 , -0.1000304 , ..., -0.1953064 ,
         0.07226377,  0.27778935]], dtype=float32)